In [1]:
# Peak

In [2]:
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
#Working on plotly
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [3]:
df=pd.read_csv(r"C:\Users\2000139\ShurovoData_2018H2\ENV3_1.txt")
df.head()

,Number of points: 8192
0,Sampling rate(Hz): 1562.5
1,Time range: From: 07/01/2018 08:00:12; Until: ...
2,Units: g
3,01.07.2018 08:00:12.7250048\t0.776226
4,01.07.2018 08:00:12.7256448\t0.8002694


In [4]:
df.columns=['ENV3_1']
df = pd.DataFrame(df['ENV3_1'].str.split('\t',2).tolist())

In [5]:
df.dropna(inplace=True)
df[0]=pd.to_datetime(df[0])
df.index=df[0]
df.index=df.index.date
df.drop(columns=[0],inplace=True)
col_name = 'ENV3_1'
df.columns=[col_name]
df.isnull().sum()
df=df.astype({col_name:float,})
df.info()
p2p_df=df.groupby(df.index).agg({col_name:['min','max']})
p2p_df
header = np.array(['Min','Max'])
p2p_df.columns=header
print("Pre-Processing is done")

<class 'pandas.core.frame.DataFrame'>
Index: 1893243 entries, 2018-01-07 to 2018-01-12
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ENV3_1  float64
dtypes: float64(1)
memory usage: 28.9+ MB
Pre-Processing is done


In [6]:
p2p_df.head()

,Min,Max
2018-01-07,0.274278,1.709842
2018-01-09,0.226191,1.693937
2018-01-10,0.379328,1.679881
2018-01-12,0.321254,1.635494
2018-02-07,0.314966,1.599614


In [7]:
p2p_df['P2P'] = p2p_df.Max - p2p_df.Min
p2p_df.head()

,Min,Max,P2P
2018-01-07,0.274278,1.709842,1.435564
2018-01-09,0.226191,1.693937,1.467746
2018-01-10,0.379328,1.679881,1.300553
2018-01-12,0.321254,1.635494,1.314240
2018-02-07,0.314966,1.599614,1.284648


In [8]:
outlier_df=p2p_df[p2p_df['Max']<0.1]
outlier_df

,Min,Max,P2P
2018-09-14,0.010172,0.012392,0.002219


In [9]:
final_df_1=p2p_df[:]
final_df_1

,Min,Max,P2P
2018-01-07,0.274278,1.709842,1.435564
2018-01-09,0.226191,1.693937,1.467746
2018-01-10,0.379328,1.679881,1.300553
2018-01-12,0.321254,1.635494,1.314240
2018-02-07,0.314966,1.599614,1.284648
...,...,...,...
2018-11-25,0.299431,1.619588,1.320157
2018-11-26,0.348627,1.651769,1.303142
2018-11-27,0.360093,1.659907,1.299814
2018-12-07,0.350106,1.698376,1.348270


In [10]:
final_df_1.drop(columns=['Min','P2P'],inplace=True)
final_df_1

,Max
2018-01-07,1.709842
2018-01-09,1.693937
2018-01-10,1.679881
2018-01-12,1.635494
2018-02-07,1.599614
...,...
2018-11-25,1.619588
2018-11-26,1.651769
2018-11-27,1.659907
2018-12-07,1.698376


In [11]:
final_df_1.columns=['Peak']
final_df_1.index=final_df_1.index.astype('str')
final_df_1.drop(final_df_1[final_df_1['Peak'] <0.1].index, inplace= True)
final_df_1.index=pd.to_datetime(final_df_1.index)

In [12]:
fit1 = SimpleExpSmoothing(final_df_1['Peak']).fit(smoothing_level=0.2,optimized = False)
final_df_1['Smooth Peak']=list(fit1.fittedvalues)

C:\Users\2000139\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [13]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=final_df_1.index,y=final_df_1['Smooth Peak'],mode='lines',name = 'Smooth_Peak',line=dict(color="#ffae42")))
fig.add_trace(go.Scatter(x=outlier_df.index,y=outlier_df.Max,mode='markers',name = 'Non Running',line=dict(color="#ff0000")))
fig.add_trace(go.Scatter(x=final_df_1.index,y=final_df_1.Peak,mode='markers',name = 'Peak',line=dict(color="#5dd55d")))
fig.update_layout(hovermode="x")
fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'PEAK DATA','xanchor':'center','x':0.5})
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Amplitude')

In [14]:
#p2p_df.to_csv(r'..\ShurovoData_2018H2\P2P_output\final_P2P.csv',header=True) 

In [15]:
fig=go.Figure()
#fig.add_trace(go.Scatter(x=RMS_copy.index,y=RMS_copy.RMS,name='RMS'))
scatter = px.scatter(p2p_df.Max,labels={'x':'Date','y':'Value'})
scatter

In [16]:
scatter.write_html(r'..\ShurovoData_2018H2\P2P_output\Scatter\Scatter.html')

FileNotFoundError: [Errno 2] No such file or directory: '..\\ShurovoData_2018H2\\P2P_output\\Scatter\\Scatter.html'

In [ ]:
fig=go.Figure()
#fig.add_trace(go.Scatter(x=RMS_copy.index,y=RMS_copy.RMS,name='RMS'))
line = px.line(p2p_df.P2P,labels={'x':'Date','y':'Value'})
line

In [ ]:
line.write_html(r'..\ShurovoData_2018H2\P2P_output\Line\Line.html')

In [ ]:
# df['date'] = df.index.astype('str')

In [ ]:
# peak_val = [df.loc[df.date == '2018-01-09']['DISPL_X_MAX_T_1']]
# peak_val[0]



In [ ]:
# peak_val = list(df.loc[df.date == '2018-01-09']['DISPL_X_MAX_T_1'])
# peak_val

In [ ]:
# # finding peaks
# from scipy.signal import find_peaks
# lst = peak_val
# peaks, _ = find_peaks(lst, height=0)

# peak_values = []
# for i in peaks:
#     peak_values.append(lst[i])
    
# print("Min peak "+str(min(peak_values)))